In [25]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.tree import DecisionTreeClassifier, export_text
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.metrics import accuracy_score, silhouette_score
from sklearn.inspection import permutation_importance
import plotly.express as px
import plotly.graph_objects as go
from dash import Dash, dcc, html, Input, Output

# Генерация синтетических данных
np.random.seed(42)
n_samples = 200
X, y = make_classification(n_samples=n_samples, n_features=5, n_informative=3, n_classes=2, random_state=42)
X = pd.DataFrame(X, columns=['возраст', 'MMSE', 'Aβ42', 'EEG_delta', 'EEG_theta'])
X['возраст'] = (X['возраст'] * 10 + 70).clip(50, 90)
X['MMSE'] = (X['MMSE'] * 3 + 22).clip(15, 30)
X['Aβ42'] = (X['Aβ42'] * 100 + 800).clip(400, 1200)
X['EEG_delta'] = (X['EEG_delta'] + 1) / 2
X['EEG_theta'] = (X['EEG_theta'] + 1) / 2
y = pd.Series(y, name='AD')
y_reg = X['Aβ42'] + np.random.normal(0, 50, n_samples)
X_train, X_test, y_train, y_test, y_reg_train, y_reg_test = train_test_split(X, y, y_reg, test_size=0.3, random_state=42)

# Инициализация Dash
app = Dash(__name__)

# Макет
app.layout = html.Div([
    html.H1('Интерактивный виджет для моделей деменции'),
    html.H3('Более сложные модели часто имеют меньшую интерпретируемость, но бóльшую точность.'),
    html.H3('Предикторы в данных:'),
    html.P('Возраст: возраст пациента (50–90 лет).'),
    html.P('MMSE: оценка когнитивных функций (15–30 баллов, ниже — хуже).'),
    html.P('Aβ42: уровень амилоид-бета 42 в спинномозговой жидкости (400–1200, маркер болезни Альцгеймера).'),
    html.P('EEG_delta: доля дельта-волн в ЭЭГ (0–1, низкочастотная активность мозга).'),
    html.P('EEG_theta: доля тета-волн в ЭЭГ (0–1, связана с расслаблением/патологией).'),
    html.P('Целевая переменная: AD (0 = Здоров, 1 = болезнь Альцгеймера).'),
    html.P('Для регрессии: y_reg — синтетический уровень Aβ42 с шумом.'),
    dcc.Tabs([
        dcc.Tab(label='Линейная регрессия', children=[
            html.H3('Степень полинома'),
            dcc.Slider(id='linreg-degree', min=1, max=5, step=1, value=1, marks={i: str(i) for i in range(1, 6)}),
            dcc.Graph(id='linreg-plot'),
            html.Div(id='linreg-formula', style={'whiteSpace': 'pre-wrap'})
        ]),
        dcc.Tab(label='Логистическая регрессия', children=[
            html.H3('Количество переменных'),
            dcc.Slider(id='logreg-n-vars', min=1, max=5, step=1, value=2, marks={1: '1', 2: '2', 3: '3', 4: '4', 5: '5'}),
            dcc.Graph(id='logreg-plot'),
            html.Div(id='logreg-formula', style={'whiteSpace': 'pre-wrap'})
        ]),
        dcc.Tab(label='Дерево решений', children=[
            html.H3('Глубина дерева'),
            dcc.Slider(id='tree-depth', min=2, max=10, step=1, value=3, marks={i: str(i) for i in range(2, 11)}),
            dcc.Graph(id='tree-plot'),
            html.Div(id='tree-formula', style={'whiteSpace': 'pre-wrap'})
        ]),
        dcc.Tab(label='Случайный лес', children=[
            html.H3('Количество деревьев'),
            dcc.Slider(id='rf-n-estimators', min=10, max=200, step=10, value=50, marks={i: str(i) for i in range(10, 201, 50)}),
            html.H3('Максимальная глубина'),
            dcc.Slider(id='rf-max-depth', min=2, max=10, step=1, value=3, marks={i: str(i) for i in range(2, 11)}),
            dcc.Graph(id='rf-plot'),
            dcc.Graph(id='rf-importance-plot'),
            html.Div(id='rf-formula', style={'whiteSpace': 'pre-wrap'})
        ]),
        dcc.Tab(label='SVM', children=[
            html.H3('Ядро'),
            dcc.Dropdown(id='svm-kernel', options=[{'label': 'Линейное', 'value': 'linear'}, {'label': 'RBF', 'value': 'rbf'}], value='rbf'),
            html.H3('Параметр C'),
            dcc.Slider(id='svm-c', min=0.001, max=100, step=0.1, value=1, marks={0.001: '0.001', 1: '1', 10: '10', 50: '50', 100: '100'}),
            html.Div(id='svm-gamma-container', style={'display': 'block'}, children=[
                html.H3('Параметр gamma'),
                dcc.Slider(id='svm-gamma', min=0.001, max=10, step=0.1, value=1, marks={i: str(i) for i in range(1, 11)})
            ]),
            html.H3('Количество признаков для визуализации'),
            dcc.Slider(id='svm-n-features', min=2, max=5, step=1, value=2, marks={2: '2D', 3: '3D', 4: '4D', 5: '5D'}),
            dcc.Graph(id='svm-plot'),
            html.Div(id='svm-formula', style={'whiteSpace': 'pre-wrap'})
        ]),
        dcc.Tab(label='Нейросети', children=[
            html.H3('Количество нейронов'),
            dcc.Slider(id='nn-neurons', min=10, max=100, step=10, value=10, marks={i: str(i) for i in range(10, 101, 20)}),
            html.H3('Количество слоёв'),
            dcc.Slider(id='nn-layers', min=1, max=3, step=1, value=1, marks={1: '1', 2: '2', 3: '3'}),
            html.H3('Активация'),
            dcc.Dropdown(id='nn-activation', options=[{'label': 'ReLU', 'value': 'relu'}, {'label': 'Tanh', 'value': 'tanh'}], value='relu'),
            dcc.Graph(id='nn-plot'),
            html.Div(id='nn-formula', style={'whiteSpace': 'pre-wrap'})
        ]),
        dcc.Tab(label='Гауссовская смесь', children=[
            html.H3('Количество компонент'),
            dcc.Slider(id='gmm-components', min=2, max=5, step=1, value=2, marks={i: str(i) for i in range(2, 6)}),
            dcc.Graph(id='gmm-plot'),
            html.Div(id='gmm-formula', style={'whiteSpace': 'pre-wrap'})
        ]),
        dcc.Tab(label='kNN', children=[
            html.H3('Количество соседей'),
            dcc.Slider(id='knn-k', min=3, max=15, step=1, value=5, marks={i: str(i) for i in range(3, 16, 3)}),
            dcc.Graph(id='knn-plot'),
            html.Div(id='knn-formula', style={'whiteSpace': 'pre-wrap'})
        ]),
        dcc.Tab(label='k-means', children=[
            html.H3('Количество кластеров'),
            dcc.Slider(id='kmeans-clusters', min=2, max=5, step=1, value=2, marks={i: str(i) for i in range(2, 6)}),
            dcc.Graph(id='kmeans-plot'),
            html.Div(id='kmeans-formula', style={'whiteSpace': 'pre-wrap'})
        ])
    ])
])

# Callback для линейной регрессии
@app.callback(
    [Output('linreg-plot', 'figure'), Output('linreg-formula', 'children')],
    Input('linreg-degree', 'value')
)
def update_linreg(degree):
    poly = PolynomialFeatures(degree=degree)
    X_poly_train = poly.fit_transform(X_train[['возраст', 'MMSE']])
    X_poly_test = poly.transform(X_test[['возраст', 'MMSE']])
    model = LinearRegression().fit(X_poly_train, y_reg_train)
    r2 = model.score(X_poly_test, y_reg_test) * 100
    formula = f'y = {model.intercept_:.1f} + ' + ' + '.join(f'{coef:.1f}*{name}' for coef, name in zip(model.coef_, poly.get_feature_names_out(['возраст', 'MMSE'])))
    
    X_range = np.linspace(X_test['возраст'].min(), X_test['возраст'].max(), 100)
    X_poly_range = poly.transform(pd.DataFrame({'возраст': X_range, 'MMSE': X_test['MMSE'].mean()}))
    y_pred = model.predict(X_poly_range)
    fig = px.scatter(x=X_test['возраст'], y=y_reg_test, labels={'x': 'Возраст', 'y': 'Aβ42'})
    fig.add_trace(go.Scatter(x=X_range, y=y_pred, mode='lines', name=f'Полином степени {degree}'))
    
    return fig, f'Линейная регрессия (степень полинома = {degree})\nФормула: {formula}\nR²: {r2:.2f}%'

# Callback для логистической регрессии
@app.callback(
    [Output('logreg-plot', 'figure'), Output('logreg-formula', 'children')],
    Input('logreg-n-vars', 'value')
)
def update_logreg(n_vars):
    features = ['возраст', 'MMSE', 'Aβ42', 'EEG_delta', 'EEG_theta'][:n_vars]
    model = LogisticRegression(max_iter=1000).fit(X_train[features], y_train)
    acc = accuracy_score(y_test, model.predict(X_test[features])) * 100
    formula = f'logit(P) = {model.intercept_[0]:.1f} + ' + ' + '.join(f'{coef:.1f}*{name}' for coef, name in zip(model.coef_[0], features))
    selected_vars = ', '.join(features)
    
    x_range = np.linspace(X_test['возраст'].min(), X_test['возраст'].max(), 100)
    y_range = np.linspace(X_test['MMSE'].min(), X_test['MMSE'].max(), 100)
    X_grid, Y_grid = np.meshgrid(x_range, y_range)
    X_grid_df = pd.DataFrame(np.c_[X_grid.ravel(), Y_grid.ravel()], columns=['возраст', 'MMSE'])
    for f in features[2:]:
        X_grid_df[f] = X_test[f].mean()
    Z = model.predict_proba(X_grid_df[features])[:, 1].reshape(X_grid.shape)
    fig = px.scatter(x=X_test['возраст'], y=X_test['MMSE'], color=list(y_test.map({0: 'Здоров', 1: 'AD'})), 
                     color_discrete_map={'Здоров': 'blue', 'AD': 'red'}, labels={'x': 'Возраст', 'y': 'MMSE'})
    fig.add_trace(go.Contour(x=x_range, y=y_range, z=Z, contours=dict(start=0.5, end=0.5, size=0.1), showscale=False))
    
    return fig, f'Логистическая регрессия (переменных = {n_vars})\nВыбранные переменные: {selected_vars}\nФормула: {formula}\nТочность: {acc:.2f}%'

# Callback для дерева решений
@app.callback(
    [Output('tree-plot', 'figure'), Output('tree-formula', 'children')],
    Input('tree-depth', 'value')
)
def update_tree(depth):
    model = DecisionTreeClassifier(max_depth=depth, random_state=42).fit(X_train[['возраст', 'MMSE']], y_train)
    acc = accuracy_score(y_test, model.predict(X_test[['возраст', 'MMSE']])) * 100
    tree_text = export_text(model, feature_names=['возраст', 'MMSE'], decimals=1)
    tree_text = tree_text.replace('class: 0', 'Класс: Здоров').replace('class: 1', 'Класс: AD')
    
    x_range = np.linspace(X_test['возраст'].min(), X_test['возраст'].max(), 100)
    y_range = np.linspace(X_test['MMSE'].min(), X_test['MMSE'].max(), 100)
    X_grid, Y_grid = np.meshgrid(x_range, y_range)
    X_grid_df = pd.DataFrame(np.c_[X_grid.ravel(), Y_grid.ravel()], columns=['возраст', 'MMSE'])
    Z = model.predict(X_grid_df).reshape(X_grid.shape)
    fig = px.scatter(x=X_test['возраст'], y=X_test['MMSE'], color=list(y_test.map({0: 'Здоров', 1: 'AD'})), 
                     color_discrete_map={'Здоров': 'blue', 'AD': 'red'}, labels={'x': 'Возраст', 'y': 'MMSE'})
    fig.add_trace(go.Contour(x=x_range, y=y_range, z=Z, showscale=False))
    
    return fig, f'Дерево решений (глубина = {depth})\nПравила дерева:\n{tree_text}\nТочность: {acc:.2f}%'

# Callback для случайного леса
@app.callback(
    [Output('rf-plot', 'figure'), Output('rf-importance-plot', 'figure'), Output('rf-formula', 'children')],
    [Input('rf-n-estimators', 'value'), Input('rf-max-depth', 'value')]
)
def update_rf(n_estimators, max_depth):
    model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42).fit(X_train, y_train)
    acc = accuracy_score(y_test, model.predict(X_test)) * 100
    importance = pd.DataFrame({'Признак': X.columns, 'Важность': model.feature_importances_}).sort_values('Важность', ascending=False)
    importance_text = '\n'.join(f'{row["Признак"]}: {row["Важность"]:.2f}' for _, row in importance.iterrows())
    
    max_trees_to_show = min(n_estimators, 5)
    ascii_trees = []
    for i in range(max_trees_to_show):
        tree_depth = np.random.randint(2, max_depth + 1)
        tree_str = "- " + " ".join(["* -" for _ in range(tree_depth - 1)]) + " *"
        ascii_trees.append(f'Дерево {i + 1}: {tree_str}')
    ascii_text = '\n'.join(ascii_trees)
    if n_estimators > max_trees_to_show:
        ascii_text += f'\n...и ещё {n_estimators - max_trees_to_show} деревьев'
    ascii_text += f'\nКаждое дерево разбивает данные на узлы (*). При большей глубине (max_depth) деревья длиннее и сложнее.\nПри большем числе деревьев (n_estimators) лес "гуще", но сложнее интерпретировать.'
    
    example_tree = model.estimators_[0]
    tree_text = export_text(example_tree, feature_names=list(X.columns), decimals=1)
    tree_text = tree_text.replace('class: 0', 'Класс: Здоров').replace('class: 1', 'Класс: AD')
    
    x_range = np.linspace(X_test['возраст'].min(), X_test['возраст'].max(), 100)
    y_range = np.linspace(X_test['MMSE'].min(), X_test['MMSE'].max(), 100)
    X_grid, Y_grid = np.meshgrid(x_range, y_range)
    X_grid_df = pd.DataFrame(np.c_[X_grid.ravel(), Y_grid.ravel()], columns=['возраст', 'MMSE'])
    for col in X.columns[2:]:
        X_grid_df[col] = X_test[col].mean()
    Z = model.predict(X_grid_df).reshape(X_grid.shape)
    fig = px.scatter(x=X_test['возраст'], y=X_test['MMSE'], color=list(y_test.map({0: 'Здоров', 1: 'AD'})), 
                     color_discrete_map={'Здоров': 'blue', 'AD': 'red'}, labels={'x': 'Возраст', 'y': 'MMSE'})
    fig.add_trace(go.Contour(x=x_range, y=y_range, z=Z, showscale=False))
    
    importance_fig = px.bar(importance, x='Важность', y='Признак', orientation='h', labels={'Важность': 'Важность', 'Признак': 'Признак'})
    
    return fig, importance_fig, f'Случайный лес\nКоличество деревьев: {n_estimators}\nМаксимальная глубина: {max_depth}\n\nВажность признаков:\n{importance_text}\n\nASCII-схема леса:\n{ascii_text}\n\nПример одного дерева:\n{tree_text}\n\nТочность: {acc:.2f}%'

# Callback для SVM
@app.callback(
    [Output('svm-plot', 'figure'), Output('svm-formula', 'children'), Output('svm-gamma-container', 'style')],
    [Input('svm-kernel', 'value'), Input('svm-c', 'value'), Input('svm-gamma', 'value'), Input('svm-n-features', 'value')]
)
def update_svm(kernel, c, gamma, n_features):
    features = ['возраст', 'MMSE', 'Aβ42', 'EEG_delta', 'EEG_theta'][:n_features]
    if kernel == 'rbf':
        model = SVC(C=c, kernel=kernel, gamma=gamma, probability=True).fit(X_train[features], y_train)
        style = {'display': 'block'}
    else:
        model = SVC(C=c, kernel=kernel, probability=True).fit(X_train[features], y_train)
        style = {'display': 'none'}
    acc = accuracy_score(y_test, model.predict(X_test[features])) * 100
    n_vectors = len(model.support_)
    
    if kernel == 'linear':
        weights = pd.DataFrame({'Признак': features, 'Вес': model.coef_[0]}).sort_values('Вес', ascending=False)
        importance_text = '\n'.join(f'{row["Признак"]}: {row["Вес"]:.2f}' for _, row in weights.iterrows())
    else:
        perm_importance = permutation_importance(model, X_test[features], y_test, n_repeats=10, random_state=42)
        importance = pd.DataFrame({'Признак': features, 'Важность': perm_importance.importances_mean}).sort_values('Важность', ascending=False)
        importance_text = '\n'.join(f'{row["Признак"]}: {row["Важность"]:.2f}' for _, row in importance.iterrows())
    
    if n_features == 2:
        x_range = np.linspace(X_test['возраст'].min(), X_test['возраст'].max(), 100)
        y_range = np.linspace(X_test['MMSE'].min(), X_test['MMSE'].max(), 100)
        X_grid, Y_grid = np.meshgrid(x_range, y_range)
        X_grid_df = pd.DataFrame(np.c_[X_grid.ravel(), Y_grid.ravel()], columns=['возраст', 'MMSE'])
        for col in features[2:]:
            X_grid_df[col] = X_test[col].mean() if col in X_test else 0
        Z = model.predict(X_grid_df).reshape(X_grid.shape)
        fig = px.scatter(x=X_test['возраст'], y=X_test['MMSE'], color=list(y_test.map({0: 'Здоров', 1: 'AD'})), 
                         color_discrete_map={'Здоров': 'blue', 'AD': 'red'}, labels={'x': 'Возраст', 'y': 'MMSE'})
        fig.add_trace(go.Contour(x=x_range, y=y_range, z=Z, showscale=False))
        sv = model.support_vectors_
        fig.add_trace(go.Scatter(x=sv[:, 0], y=sv[:, 1], mode='markers', marker=dict(color='black', size=10, line=dict(width=2, color='DarkSlateGrey')), name='Опорные вектора'))
    elif n_features == 3:
        x_range = np.linspace(X_test['возраст'].min(), X_test['возраст'].max(), 50)
        y_range = np.linspace(X_test['MMSE'].min(), X_test['MMSE'].max(), 50)
        z_range = np.linspace(X_test['Aβ42'].min(), X_test['Aβ42'].max(), 50)
        X_grid, Y_grid, Z_grid = np.meshgrid(x_range, y_range, z_range)
        X_grid_df = pd.DataFrame(np.c_[X_grid.ravel(), Y_grid.ravel(), Z_grid.ravel()], columns=['возраст', 'MMSE', 'Aβ42'])
        for col in features[3:]:
            X_grid_df[col] = X_test[col].mean() if col in X_test else 0
        preds = model.decision_function(X_grid_df).reshape(X_grid.shape)
        fig = px.scatter_3d(x=X_test['возраст'], y=X_test['MMSE'], z=X_test['Aβ42'], 
                            color=list(y_test.map({0: 'Здоров', 1: 'AD'})), 
                            color_discrete_map={'Здоров': 'blue', 'AD': 'red'}, 
                            labels={'x': 'Возраст', 'y': 'MMSE', 'z': 'Aβ42'})
        fig.add_trace(go.Isosurface(x=x_range, y=y_range, z=z_range, value=preds, isomin=-1, isomax=1, 
                                   surface_count=1, caps=dict(x_show=False, y_show=False, z_show=False), 
                                   name='Разделяющая поверхность', colorscale=[[0, 'gray'], [1, 'gray']], 
                                   showscale=False, opacity=1, showlegend=False))
        sv = model.support_vectors_[:, :3]
        fig.add_trace(go.Scatter3d(x=sv[:, 0], y=sv[:, 1], z=sv[:, 2], mode='markers', 
                                   marker=dict(color='black', size=5, line=dict(width=2, color='DarkSlateGrey')), 
                                   name='Опорные вектора'))
        fig.update_layout(showlegend=True, legend=dict(title='Классы'), coloraxis_showscale=False)
        fig.update_traces(colorbar=None, selector=dict(type='isosurface'))
        fig.update_coloraxes(showscale=False)
    else:
        fig = go.Figure()
        fig.add_annotation(text='График невозможен из-за высокой размерности (>3 признаков)', x=0.5, y=0.5, showarrow=False)
    
    return fig, f'SVM\nЯдро: {kernel}\nПараметр C: {c}\nGamma: {gamma if kernel == "rbf" else "N/A"}\nКоличество опорных векторов: {n_vectors}\n\nВажность признаков:\n{importance_text}\n\nТочность: {acc:.2f}%', style

# Callback для нейросети
@app.callback(
    [Output('nn-plot', 'figure'), Output('nn-formula', 'children')],
    [Input('nn-neurons', 'value'), Input('nn-layers', 'value'), Input('nn-activation', 'value')]
)
def update_nn(neurons, layers, activation):
    hidden_layers = (neurons,) * layers
    model = MLPClassifier(hidden_layer_sizes=hidden_layers, activation=activation, max_iter=1000, random_state=42).fit(X_train, y_train)
    acc = accuracy_score(y_test, model.predict(X_test)) * 100
    formula = f'Нейросеть\nКоличество слоёв: {layers}\nНейронов на слой: {neurons}\nАктивация: {activation}\nПри большом числе нейронов и слоёв модель сложнее, но может переобучаться'
    
    x_range = np.linspace(X_test['возраст'].min(), X_test['возраст'].max(), 100)
    y_range = np.linspace(X_test['MMSE'].min(), X_test['MMSE'].max(), 100)
    X_grid, Y_grid = np.meshgrid(x_range, y_range)
    X_grid_df = pd.DataFrame(np.c_[X_grid.ravel(), Y_grid.ravel()], columns=['возраст', 'MMSE'])
    for col in X.columns[2:]:
        X_grid_df[col] = X_test[col].mean()
    Z = model.predict_proba(X_grid_df)[:, 1].reshape(X_grid.shape)
    fig = px.scatter(x=X_test['возраст'], y=X_test['MMSE'], color=list(y_test.map({0: 'Здоров', 1: 'AD'})), 
                     color_discrete_map={'Здоров': 'blue', 'AD': 'red'}, labels={'x': 'Возраст', 'y': 'MMSE'})
    fig.add_trace(go.Contour(x=x_range, y=y_range, z=Z, showscale=False))
    probs = model.predict_proba(X_test)[:, 1]
    fig.update_traces(marker=dict(size=12 * probs, line=dict(width=2, color='DarkSlateGrey')), selector=dict(mode='markers'))
    
    perm_importance = permutation_importance(model, X_test, y_test, n_repeats=10, random_state=42)
    importance = pd.DataFrame({'Признак': X.columns, 'Важность': perm_importance.importances_mean}).sort_values('Важность', ascending=False)
    importance_text = '\n'.join(f'{row["Признак"]}: {row["Важность"]:.2f}' for _, row in importance.iterrows())
    
    return fig, f'{formula}\n\nВажность признаков:\n{importance_text}\n\nТочность: {acc:.2f}%'

# Callback для GMM
@app.callback(
    [Output('gmm-plot', 'figure'), Output('gmm-formula', 'children')],
    Input('gmm-components', 'value')
)
def update_gmm(components):
    model = GaussianMixture(n_components=components, random_state=42).fit(X_train[['возраст', 'MMSE']])
    labels = model.predict(X_test[['возраст', 'MMSE']])
    silhouette = silhouette_score(X_test[['возраст', 'MMSE']], labels) * 100
    cluster_sizes = np.bincount(labels)
    cluster_text = '\n'.join(f'Кластер {i}: {size} точек' for i, size in enumerate(cluster_sizes))
    
    point = X_test[['возраст', 'MMSE']].iloc[0]
    probs = model.predict_proba(pd.DataFrame([point], columns=['возраст', 'MMSE']))[0]
    probs_text = f'Пример пациента (возраст={point["возраст"]:.1f}, MMSE={point["MMSE"]:.1f}):\nВероятности: ' + ', '.join(f'Кластер {i}: {p:.0%}' for i, p in enumerate(probs))
    
    # Добавляем кросс-табуляцию
    crosstab = pd.crosstab(labels, y_test.map({0: 'Здоров', 1: 'AD'}))
    crosstab_text = f'Соответствие кластеров меткам:\n{crosstab.to_string()}'
    
    fig = px.scatter(x=X_test['возраст'], y=X_test['MMSE'], color=labels.astype(str), labels={'x': 'Возраст', 'y': 'MMSE'})
    
    return fig, f'Гауссовская смесь\nКоличество компонент: {components}\n\nРазмеры кластеров:\n{cluster_text}\n\n{probs_text}\n\n{crosstab_text}\n\nСилуэтный коэффициент: {silhouette:.2f}%\n(высокий - хорошее разделение, низкий - перекрытие)\nПри большом числе компонент кластеры сложнее интерпретировать'
    
# Callback для kNN
@app.callback(
    [Output('knn-plot', 'figure'), Output('knn-formula', 'children')],
    Input('knn-k', 'value')
)
def update_knn(k):
    model = KNeighborsClassifier(n_neighbors=k).fit(X_train[['возраст', 'MMSE']], y_train)
    acc = accuracy_score(y_test, model.predict(X_test[['возраст', 'MMSE']])) * 100
    
    point = X_test[['возраст', 'MMSE']].iloc[0]
    point_df = pd.DataFrame([point], columns=['возраст', 'MMSE'])
    distances, indices = model.kneighbors(point_df)
    neighbors_classes = y_train.iloc[indices[0]].map({0: 'Здоров', 1: 'AD'}).tolist()
    neighbors_text = f'Пример пациента (возраст={point["возраст"]:.1f}, MMSE={point["MMSE"]:.1f}):\nСоседи: ' + ', '.join(f'{cls} (расст. {d:.1f})' for cls, d in zip(neighbors_classes, distances[0]))
    probs = model.predict_proba(point_df)[0]
    prob_text = f'Вероятность AD: {probs[1]:.0%}, Здоров: {probs[0]:.0%}'
    
    x_range = np.linspace(X_test['возраст'].min(), X_test['возраст'].max(), 100)
    y_range = np.linspace(X_test['MMSE'].min(), X_test['MMSE'].max(), 100)
    X_grid, Y_grid = np.meshgrid(x_range, y_range)
    X_grid_df = pd.DataFrame(np.c_[X_grid.ravel(), Y_grid.ravel()], columns=['возраст', 'MMSE'])
    Z = model.predict(X_grid_df).reshape(X_grid.shape)
    fig = px.scatter(x=X_test['возраст'], y=X_test['MMSE'], color=list(y_test.map({0: 'Здоров', 1: 'AD'})), 
                     color_discrete_map={'Здоров': 'blue', 'AD': 'red'}, labels={'x': 'Возраст', 'y': 'MMSE'})
    fig.add_trace(go.Contour(x=x_range, y=y_range, z=Z, showscale=False))
    
    return fig, f'Классификация по {k} ближайшим соседям\n\n{neighbors_text}\n\n{prob_text}\n\nПри малом k решение простое, при большом k сложнее из-за большего числа соседей\n\nТочность: {acc:.2f}%'

# Callback для k-means
@app.callback(
    [Output('kmeans-plot', 'figure'), Output('kmeans-formula', 'children')],
    Input('kmeans-clusters', 'value')
)
def update_kmeans(clusters):
    model = KMeans(n_clusters=clusters, random_state=42).fit(X_train[['возраст', 'MMSE']])
    labels = model.predict(X_test[['возраст', 'MMSE']])
    silhouette = silhouette_score(X_test[['возраст', 'MMSE']], labels) * 100
    cluster_sizes = np.bincount(labels)
    cluster_text = '\n'.join(f'Кластер {i}: {size} точек' for i, size in enumerate(cluster_sizes))
    
    point = X_test[['возраст', 'MMSE']].iloc[0]
    distances = model.transform(pd.DataFrame([point], columns=['возраст', 'MMSE']))[0]
    closest_cluster = np.argmin(distances)
    dist_text = f'Пример пациента (возраст={point["возраст"]:.1f}, MMSE={point["MMSE"]:.1f}):\nБлижайший кластер: {closest_cluster} (расст. {distances[closest_cluster]:.1f})'
    
    # Добавляем кросс-табуляцию
    crosstab = pd.crosstab(labels, y_test.map({0: 'Здоров', 1: 'AD'}))
    crosstab_text = f'Соответствие кластеров меткам:\n{crosstab.to_string()}'
    
    fig = px.scatter(x=X_test['возраст'], y=X_test['MMSE'], color=labels.astype(str), labels={'x': 'Возраст', 'y': 'MMSE'})
    
    return fig, f'Кластеризация k-means\nКоличество кластеров: {clusters}\n\nРазмеры кластеров:\n{cluster_text}\n\n{dist_text}\n\n{crosstab_text}\n\nСилуэтный коэффициент: {silhouette:.2f}%\n(высокий - хорошее разделение, низкий - перекрытие)\nПри большом числе кластеров интерпретируемость снижается'
    
# Запуск приложения
if __name__ == '__main__':
    app.run(debug=True, port=8051)